# Reddit stock market prediction

### Import libraries

In [2]:
# pull libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

### Preprocess data

In [3]:
# pull data
djia_data = pd.read_csv('./stocknews/Combined_News_DJIA.csv', parse_dates=True)

djia_data['combined'] = djia_data[djia_data.columns[2:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)

def sanitise_row(row):
    return re.sub('[^A-Za-z ]+', '', row.replace("b\"", "").replace("b'", ""))

def remove_stop_words(sentence):
    sentence = sanitise_row(sentence)
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

djia_data['combined'] = djia_data['combined'].map(lambda x: remove_stop_words(x))

# print(type(djia_data['combined']))

In [3]:
# Ratio of label 1s and 0s
data_label = np.array(djia_data['Label'])

print(np.sum(data_label == 1) / data_label.shape[0])

0.5354449472096531


### Preprocess data

In [5]:
# prepare train and test data
all_data = djia_data['combined']
data_label = np.array(djia_data['Label'])

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(all_data)
# print(vectorizer.get_feature_names())
# print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    all_data, data_label, test_size=0.80, random_state=23)
# we also need dictionary that maps word to number of occurences -> 
# to handle cases where word is not found in the training corpus

In [6]:
# We need to convert training data into a vector for LSTM
# how does the vector look like?
# need to build word_index first!


# generate word numbers
def gen_word_indexes(data): 
    word_index = {}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2  # unknown
    word_index["<UNUSED>"] = 3

    index = 4
    # loop through everything in all_data
    for row in data:
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                word_index[word] = index
                index += 1

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return (word_index, reverse_word_index)


# text --> numbers corresponding to words
def vectorise_text_data(data, word_index):
    vectorised_data = []
    for row in data:
        current_row = []
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                current_row.append(word_index["<UNUSED>"])
            else:
                current_row.append(word_index[word])
        vectorised_data.append(current_row)
    return vectorised_data

def aggregate_previous_days(num_days, data):
    new = data.copy()
    for i in range(num_days, len(data) - num_days):
        for d in range(1, num_days):
            new.at[i] = new[i] + " " + data[d+i-1]
    return new    

In [65]:
(word_index, reverse_word_index) = gen_word_indexes(all_data)
# save word_index somewhere
import pickle
pickle_out = open("word_index.pickle","wb")
pickle.dump(word_index, pickle_out)
pickle_out.close()

# X_vectorised = vectorise_text_data(all_data, word_index)
time_series_data = aggregate_previous_days(5, all_data)
X_series_vectorised = vectorise_text_data(time_series_data, word_index)

In [9]:
print(data_label)
data_label_5day = data_label.copy()

# remove the first 4 and last data label:
data_label_5day = data_label_5day[5:]
X_5day_vectorised = X_series_vectorised[4:-1]

print("sizes" + str(len(X_5day_vectorised)) + " " +  str(len(data_label_5day)))

[0 1 0 ... 1 1 1]
sizes1984 1984


In [10]:
from keras.preprocessing import sequence

# number of most-frequent words to use
nb_words = 40000
# cut texts after this number of words
maxlen = 800

# we can change the "X???_vectorised" based on the number of cumulative days:
X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(
    X_5day_vectorised, data_label_5day, test_size=0.20, random_state=32)

X_v_pad_train = sequence.pad_sequences(X_v_train,
                                       value=word_index["<PAD>"],
                                       padding='post',
                                       maxlen=maxlen)
X_v_pad_test = sequence.pad_sequences(X_v_test, 
                                      value=word_index["<PAD>"],
                                      padding='post',
                                      maxlen=maxlen)

print(X_v_pad_train.shape, X_v_pad_test.shape)
print(X_v_pad_train[6])

Using TensorFlow backend.


(1587, 800) (397, 800)
[ 5869   156  2488  1041  7434 26154  1390 12330  6580  5334  2525    58
   165   156 26155 25358 12749   650  6207  1132   625  3652 12888  3627
   571    58  5469   650  1863 26156  8893 26157  1174  8893  3652   625
  1970  3867  3420  1342 13204  4446   650  2901  2056  2971  3556  5411
  6036  1491  1492  4414    58  3759   238   285  1833   799  6625  2149
  3641  2852   767   696   939    58  1490    79    47  1586  5428    58
  1721  2927  1248  4035  4332  6650  6060  1625  7886   896 25929  1735
  1003  6405  1249  1010 17756 17757  8749  9327  6537 18935 26151  2980
   500   745    79  2244  3570   405  2765   763 12843  1877  7918 13956
    58   763 12843 17058 26158 13480 26159 26160   310   763 17914  1491
   523  4642  1912  1397   347 13480   500  3004   712  5877  5607 20055
   434   694   528  1103  2306 10081  5394 26161  4381    98 21031  1383
 25024 26162 26163  2118  1986  4430  1096  1384   474   732 10076 11463
   176  7376  6279  2512 189

Now what? We train!

### Training using Naive Bayes

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# LogisticRegression(random_state=0, solver='lbfgs')

v = CountVectorizer()

text_clf = Pipeline([
    ('vect', v),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)  

0.535175879396985

### Training using LSTM

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM

EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=EMBEDDING_DIM, input_length=maxlen))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(LSTM_DIM, dropout=0.8, recurrent_dropout=0.8))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
batch_size = 128
model.fit(X_v_pad_train, y_v_train, epochs=7, batch_size=batch_size, 
          shuffle=True, validation_split=0.3, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 1110 samples, validate on 477 samples
Epoch 1/7
1110/1110 [==============================] - 15s 14ms/step - loss: 0.6950 - acc: 0.5090 - val_loss: 0.6924 - val_acc: 0.5094
Epoch 2/7
1110/1110 [==============================] - 14s 12ms/step - loss: 0.6916 - acc: 0.5234 - val_loss: 0.6925 - val_acc: 0.5115
Epoch 3/7
1110/1110 [==============================] - 14s 13ms/step - loss: 0.6903 - acc: 0.5216 - val_loss: 0.6926 - val_acc: 0.5115
Epoch 4/7
 256/1110 [=====>........................] - ETA: 9s - loss: 0.6874 - acc: 0.5547 

In [20]:
scores = model.evaluate(X_v_pad_test, y_v_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 53.90%


In [21]:
# Save the model
from keras.models import model_from_json

#serialize to JSON
model_json = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model_json)
#serialize weights to HDF5
model.save_weights("model3.h5")
print("Saved model to the disk")

Saved model to the disk
